# Notebook 01: Adquisición de Datos y Procesamiento Inicial
**Proyecto:** Análisis SARIMAX - Starbucks Corporation (SBUX)  
**Investigador:** Frankli Zeña Zeña (UNI)

---
## Introducción
Este cuaderno tiene como objetivo la extracción de datos desde la API de **Yahoo Finance**, la limpieza de valores nulos mediante métodos de imputación financiera y la detección preliminar de anomalías de volumen. Al finalizar, los datos serán exportados a la carpeta `data/processed/` para garantizar la reproducibilidad del estudio.

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

In [2]:
ticker = "SBUX"
periodo = "5y"

df_raw = yf.download(ticker,
                     period=periodo,
                     interval="1d",
                     auto_adjust=False)

# Crear carpeta raw si no existe y guardar
os.makedirs("../data/raw", exist_ok=True)
df_raw.to_csv(f"../data/raw/{ticker}_raw.csv")

print("Datos descargados y guardados en data/raw/")
df_raw.head()

[*********************100%***********************]  1 of 1 completed

Datos descargados y guardados en data/raw/


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,SBUX,SBUX,SBUX,SBUX,SBUX,SBUX
Date,,,,,,
2021-02-16,94.238235,106.150002,106.589996,104.699997,104.879997,4928100
2021-02-17,93.114845,104.440002,105.459999,104.150002,105.110001,5733800
2021-02-18,93.587387,104.970001,105.339996,103.410004,103.779999,4098700
2021-02-19,92.160881,103.370003,105.279999,103.220001,104.750000,4128900
2021-02-22,90.770035,101.809998,102.989998,101.750000,102.989998,5211800


### Limpieza de Datos
En finanzas, los días feriados o errores de transmisión pueden generar valores nulos (`NaN`). Para no romper la continuidad de la serie de tiempo (requisito de SARIMA), aplicaremos el método **Forward Fill (ffill)**, que asume que el precio de un día sin datos es igual al último precio conocido.

In [3]:
# 1. Seleccionar columnas y limpiar
df = df_raw[['Adj Close', 'Volume']].copy()
df.columns = ['Adj Close', 'Volume']
df = df.ffill()

# 2. Detectar anomalías de volumen (Umbral: 2x la media de 20 días)
df['Vol_Avg_20'] = df['Volume'].rolling(window=20).mean()
df['Vol_Anomaly'] = df['Volume'] > (df['Vol_Avg_20'] * 2)

# 3. Calcular Retorno Logarítmico (para análisis de estacionariedad futuro)
df['Log_Return'] = np.log(df['Adj Close'] / df['Adj Close'].shift(1))

# Eliminar el primer valor NaN resultante del cálculo de retorno
df = df.dropna()

print("Procesamiento completado. Columnas generadas: Adj Close, Volume, Vol_Anomaly, Log_Return.")
df.head()

Procesamiento completado. Columnas generadas: Adj Close, Volume, Vol_Anomaly, Log_Return.


,Adj Close,Volume,Vol_Avg_20,Vol_Anomaly,Log_Return
Date,,,,,
2021-03-15,97.091217,7107300,6295740.0,False,0.010060
2021-03-16,99.266632,11376200,6618145.0,False,0.022159
2021-03-17,98.482056,10128400,6837875.0,False,-0.007935
2021-03-18,95.950020,7813400,7023610.0,False,-0.026047
2021-03-19,94.808807,21452000,7889765.0,True,-0.011965


### Visualización de Control
Graficamos el precio ajustado para verificar visualmente la tendencia y detectar posibles saltos abruptos.

In [4]:
import plotly.graph_objects as go

# Crear gráfico interactivo de Precio Ajustado
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df.index, 
    y=df['Adj Close'],
    mode="lines", 
    name="SBUX Adj Close",
    line=dict(color="#00704A", width=2) # Verde Starbucks
))

# Personalización profesional
fig.update_layout(
    title="Evolución Histórica Interactiva: Starbucks (SBUX)",
    xaxis_title="Fecha",
    yaxis_title="Precio de Cierre Ajustado (USD)",
    hovermode="x unified",
    template="plotly_white", # Fondo blanco para informes académicos
    width=1200, 
    height=500,
    xaxis=dict(rangeslider=dict(visible=True)) # Agrega un deslizador de tiempo abajo
)

fig.show()

Guardamos el DataFrame limpio en la carpeta processed/. Este será el archivo que leerán todos los notebooks siguientes.

In [5]:
os.makedirs("../data/processed", exist_ok=True)
processed_path = "../data/processed/SBUX_clean.csv"
df.to_csv(processed_path)

print(f"¡Éxito! Datos guardados en {processed_path}")
print(f"Total de registros: {len(df)}")
print(f"Total de anomalías de volumen detectadas: {df['Vol_Anomaly'].sum()}")

¡Éxito! Datos guardados en ../data/processed/SBUX_clean.csv
Total de registros: 1237
Total de anomalías de volumen detectadas: 43
